# Using Tensorflow with H2O 

This notebook shows how to use the tensorflow backend to tackle a simple image classification problem.

We start by connecting to our h2o cluster:

In [1]:
import sys, os
import h2o
from h2o.estimators.deepwater import H2ODeepWaterEstimator
import os.path
from IPython.display import Image, display, HTML
import pandas as pd
import numpy as np
import random

PATH=os.path.expanduser("~/h2o-3")
h2o.init(port=54321, nthreads=-1)
if not H2ODeepWaterEstimator.available(): exit

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_121"; Java(TM) SE Runtime Environment (build 1.8.0_121-b13); Java HotSpot(TM) 64-Bit Server VM (build 25.121-b13, mixed mode)
  Starting server from /home/arno/h2o-3/build/h2o.jar
  Ice root: /tmp/tmp7reb_75j
  JVM stdout: /tmp/tmp7reb_75j/h2o_arno_started_from_python.out
  JVM stderr: /tmp/tmp7reb_75j/h2o_arno_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster version:,3.11.0.99999
H2O cluster version age:,9 hours and 22 minutes
H2O cluster name:,H2O_from_python_arno_jsb1dh
H2O cluster total nodes:,1
H2O cluster free memory:,13.96 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [2]:
!nvidia-smi

Sat Mar 25 10:12:49 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 375.20                 Driver Version: 375.20                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 980 Ti  Off  | 0000:01:00.0      On |                  N/A |
| 13%   64C    P0    77W / 275W |   1109MiB /  6108MiB |     19%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN X (Pascal)    Off  | 0000:02:00.0     Off |                  N/A |
| 23%   35C    P8    15W / 250W |      2MiB / 12221MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [3]:
%matplotlib inline
from IPython.display import Image, display, HTML
import matplotlib.pyplot as plt

## Image Classification Task

H2O DeepWater allows you to specify a list of URIs (file paths) or URLs (links) to images, together with a response column (either a class membership (enum) or regression target (numeric)).

For this example, we use a small dataset that has a few hundred images, and three classes: cat, dog and mouse.

In [4]:
frame = h2o.import_file(PATH + "/bigdata/laptop/deepwater/imagenet/cat_dog_mouse.csv")
print(frame.dim)
print(frame.head(5))

Parse progress: |█████████████████████████████████████████████████████████| 100%
[267, 2]


C1,C2
bigdata/laptop/deepwater/imagenet/cat/102194502_49f003abd9.jpg,cat
bigdata/laptop/deepwater/imagenet/cat/11146807_00a5f35255.jpg,cat
bigdata/laptop/deepwater/imagenet/cat/1140846215_70e326f868.jpg,cat
bigdata/laptop/deepwater/imagenet/cat/114170569_6cbdf4bbdb.jpg,cat
bigdata/laptop/deepwater/imagenet/cat/1217664848_de4c7fc296.jpg,cat


To build a LeNet image classification model in H2O, simply specify `network = "lenet"` and `backend="tensorflow"` to use the our pre-built TensorFlow lenet implementation:

In [5]:
model = H2ODeepWaterEstimator(epochs=500, network = "lenet", backend="tensorflow", learning_rate=0.001)
model.train(x=[0],y=1, training_frame=frame)
model.show()

deepwater Model Build progress: |█████████████████████████████████████████| 100%
Model Details
H2ODeepWaterEstimator :  Deep Water
Model Key:  DeepWater_model_python_1490461966315_1


ModelMetricsMultinomial: deepwater
** Reported on train data. **

MSE: 9.106443439521355e-12
RMSE: 3.0176884265147975e-06
LogLoss: 1.3867613073548748e-06
Mean Per-Class Error: 0.0
Confusion Matrix: vertical: actual; across: predicted



cat,dog,mouse,Error,Rate
90.0,0.0,0.0,0.0,0 / 90
0.0,85.0,0.0,0.0,0 / 85
0.0,0.0,92.0,0.0,0 / 92
90.0,85.0,92.0,0.0,0 / 267


Top-3 Hit Ratios: 


k,hit_ratio
1,1.0
2,1.0
3,1.0


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_classification_error
,2017-03-25 10:12:51,0.000 sec,None,0.0,0,0.0,nan,nan,nan
,2017-03-25 10:12:52,2.929 sec,833 obs/sec,3.8352060,1,1024.0,0.5689077,0.8822457,0.4269663
,2017-03-25 10:12:57,7.923 sec,10879 obs/sec,253.1235955,66,67584.0,0.0000149,0.0000069,0.0
,2017-03-25 10:13:03,12.989 sec,11086 obs/sec,467.8951311,122,124928.0,0.0000037,0.0000017,0.0
,2017-03-25 10:13:03,13.708 sec,11199 obs/sec,502.4119850,131,134144.0,0.0000030,0.0000014,0.0


# DeepFeatures

We can also compute the output of any hidden layer, if we know its name.

In [6]:
model.deepfeatures(frame, "fc1/Relu")

deepfeatures progress: |██████████████████████████████████████████████████| 100%


DF.fc1/Relu.C1 DF.fc1/Relu.C2 DF.fc1/Relu.C3 DF.fc1/Relu.C4 DF.fc1/Relu.C5 DF.fc1/Relu.C6 DF.fc1/Relu.C7 DF.fc1/Relu.C8 DF.fc1/Relu.C9 DF.fc1/Relu.C10 DF.fc1/Relu.C11 DF.fc1/Relu.C12 DF.fc1/Relu.C13 DF.fc1/Relu.C14 DF.fc1/Relu.C15 DF.fc1/Relu.C16 DF.fc1/Relu.C17 DF.fc1/Relu.C18 DF.fc1/Relu.C19 DF.fc1/Relu.C20 DF.fc1/Relu.C21 DF.fc1/Relu.C22 DF.fc1/Relu.C23 DF.fc1/Relu.C24 DF.fc1/Relu.C25 DF.fc1/Relu.C26 DF.fc1/Relu.C27 DF.fc1/Relu.C28 DF.fc1/Relu.C29 DF.fc1/Relu.C30 DF.fc1/Relu.C31 DF.fc1/Relu.C32 DF.fc1/Relu.C33 DF.fc1/Relu.C34 DF.fc1/Relu.C35 DF.fc1/Relu.C36 DF.fc1/Relu.C37 DF.fc1/Relu.C38 DF.fc1/Relu.C39 DF.fc1/Relu.C40 DF.fc1/Relu.C41 DF.fc1/Relu.C42 DF.fc1/Relu.C43 DF.fc1/Relu.C44 DF.fc1/Relu.C45 DF.fc1/Relu.C46 DF.fc1/Relu.C47 DF.fc1/Relu.C48 DF.fc1/Relu.C49 DF.fc1/Relu.C50 DF.fc1/Relu.C51 DF.fc1/Relu.C52 DF.fc1/Relu.C53 DF.fc1/Relu.C54 DF.fc1/Relu.C55 DF.fc1/Relu.C56 DF.fc1/Relu.C57 DF.fc1/Relu.C58 DF.fc1/Relu.C59 DF.fc1/Relu.C60 DF.fc1/Relu.C61 DF.fc1/Relu.C62 DF.fc1/Relu.C63 DF.fc1/Relu.C64 DF.fc1/Relu.C65 DF.fc1/Relu.C66 DF.fc1/Relu.C67 DF.fc1/Relu.C68 DF.fc1/Relu.C69 DF.fc1/Relu.C70 DF.fc1/Relu.C71 DF.fc1/Relu.C72 DF.fc1/Relu.C73 DF.fc1/Relu.C74 DF.fc1/Relu.C75 DF.fc1/Relu.C76 DF.fc1/Relu.C77 DF.fc1/Relu.C78 DF.fc1/Relu.C79 DF.fc1/Relu.C80 DF.fc1/Relu.C81 DF.fc1/Relu.C82 DF.fc1/Relu.C83 DF.fc1/Relu.C84 DF.fc1/Relu.C85 DF.fc1/Relu.C86 DF.fc1/Relu.C87 DF.fc1/Relu.C88 DF.fc1/Relu.C89 DF.fc1/Relu.C90 DF.fc1/Relu.C91 DF.fc1/Relu.C92 DF.fc1/Relu.C93 DF.fc1/Relu.C94 DF.fc1/Relu.C95 DF.fc1/Relu.C96 DF.fc1/Relu.C97 DF.fc1/Relu.C98 DF.fc1/Relu.C99 DF.fc1/Relu.C100 DF.fc1/Relu.C101 DF.fc1/Relu.C102 DF.fc1/Relu.C103 DF.fc1/Relu.C104 DF.fc1/Relu.C105 DF.fc1/Relu.C106 DF.fc1/Relu.C107 DF.fc1/Relu.C108 DF.fc1/Relu.C109 DF.fc1/Relu.C110 DF.fc1/Relu.C111 DF.fc1/Relu.C112 DF.fc1/Relu.C113 DF.fc1/Relu.C114 DF.fc1/Relu.C115 DF.fc1/Relu.C116 DF.fc1/Relu.C117 DF.fc1/Relu.C118 DF.fc1/Relu.C119 DF.fc1/Relu.C120 DF.fc1/Relu.C121 DF.fc1/Relu.C122 DF.fc1/Relu.C123 DF.fc1/Relu.C124 DF.fc1/Relu.C125 DF.fc1/Relu.C126 DF.fc1/Relu.C127 DF.fc1/Relu.C128 DF.fc1/Relu.C129 DF.fc1/Relu.C130 DF.fc1/Relu.C131 DF.fc1/Relu.C132 DF.fc1/Relu.C133 DF.fc1/Relu.C134 DF.fc1/Relu.C135 DF.fc1/Relu.C136 DF.fc1/Relu.C137 DF.fc1/Relu.C138 DF.fc1/Relu.C139 DF.fc1/Relu.C140 DF.fc1/Relu.C141 DF.fc1/Relu.C142 DF.fc1/Relu.C143 DF.fc1/Relu.C144 DF.fc1/Relu.C145 DF.fc1/Relu.C146 DF.fc1/Relu.C147 DF.fc1/Relu.C148 DF.fc1/Relu.C149 DF.fc1/Relu.C150 DF.fc1/Relu.C151 DF.fc1/Relu.C152 DF.fc1/Relu.C153 DF.fc1/Relu.C154 DF.fc1/Relu.C155 DF.fc1/Relu.C156 DF.fc1/Relu.C157 DF.fc1/Relu.C158 DF.fc1/Relu.C159 DF.fc1/Relu.C160 DF.fc1/Relu.C161 DF.fc1/Relu.C162 DF.fc1/Relu.C163 DF.fc1/Relu.C164 DF.fc1/Relu.C165 DF.fc1/Relu.C166 DF.fc1/Relu.C167 DF.fc1/Relu.C168 DF.fc1/Relu.C169 DF.fc1/Relu.C170 DF.fc1/Relu.C171 DF.fc1/Relu.C172 DF.fc1/Relu.C173 DF.fc1/Relu.C174 DF.fc1/Relu.C175 DF.fc1/Relu.C176 DF.fc1/Relu.C177 DF.fc1/Relu.C178 DF.fc1/Relu.C179 DF.fc1/Relu.C180 DF.fc1/Relu.C181 DF.fc1/Relu.C182 DF.fc1/Relu.C183 DF.fc1/Relu.C184 DF.fc1/Relu.C185 DF.fc1/Relu.C186 DF.fc1/Relu.C187 DF.fc1/Relu.C188 DF.fc1/Relu.C189 DF.fc1/Relu.C190 DF.fc1/Relu.C191 DF.fc1/Relu.C192 DF.fc1/Relu.C193 DF.fc1/Relu.C194 DF.fc1/Relu.C195 DF.fc1/Relu.C196 DF.fc1/Relu.C197 DF.fc1/Relu.C198 DF.fc1/Relu.C199 DF.fc1/Relu.C200 DF.fc1/Relu.C201 DF.fc1/Relu.C202 DF.fc1/Relu.C203 DF.fc1/Relu.C204 DF.fc1/Relu.C205 DF.fc1/Relu.C206 DF.fc1/Relu.C207 DF.fc1/Relu.C208 DF.fc1/Relu.C209 DF.fc1/Relu.C210 DF.fc1/Relu.C211 DF.fc1/Relu.C212 DF.fc1/Relu.C213 DF.fc1/Relu.C214 DF.fc1/Relu.C215 DF.fc1/Relu.C216 DF.fc1/Relu.C217 DF.fc1/Relu.C218 DF.fc1/Relu.C219 DF.fc1/Relu.C220 DF.fc1/Relu.C221 DF.fc1/Relu.C222 DF.fc1/Relu.C223 DF.fc1/Relu.C224 DF.fc1/Relu.C225 DF.fc1/Relu.C226 DF.fc1/Relu.C227 DF.fc1/Relu.C228 DF.fc1/Relu.C229 DF.fc1/Relu.C230 DF.fc1/Relu.C231 DF.fc1/Relu.C232 DF.fc1/Relu.C233 DF.fc1/Relu.C234 DF.fc1/Relu.C235 DF.fc1/Relu.C236 DF.fc1/Relu.C237 DF.fc1/Relu.C238 DF.fc1/Relu.C239 DF.fc1/Relu.C240 DF.fc1/Relu.C241 DF.fc1/Relu

# Custom models

If you'd like to build your own Tensorflow network architecture, then this is easy as well.
In this example script, we are using the **Tensorflow** backend. 
Models can easily be imported/exported between H2O and Tensorflow since H2O uses Tensorflow's format for model definition.

In [7]:
def simple_model(w, h, channels, classes):
    import json
    import tensorflow as tf
    from tensorflow.python.framework import ops
    # always create a new graph inside ipython or
    # the default one will be used and can lead to
    # unexpected behavior
    graph = tf.Graph() 
    with graph.as_default():
        size = w * h * channels
        x = tf.placeholder(tf.float32, [None, size])
        W = tf.Variable(tf.zeros([size, classes]))
        b = tf.Variable(tf.zeros([classes]))
        y = tf.matmul(x, W) + b

        predictions = tf.nn.softmax(y)
        
        # labels
        y_ = tf.placeholder(tf.float32, [None, classes])
        
        # train
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_))
        train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
        
        tf.add_to_collection(ops.GraphKeys.TRAIN_OP, train_step)
        tf.add_to_collection("predictions", predictions)
        # this is required by the h2o tensorflow backend
        global_step = tf.Variable(0, name="global_step", trainable=False)
        
        init = tf.global_variables_initializer()
        tf.add_to_collection(ops.GraphKeys.INIT_OP, init.name)
        tf.add_to_collection("logits", y)
        saver = tf.train.Saver()
        meta = json.dumps({
                "inputs": {"batch_image_input": x.name, "categorical_labels": y_.name}, 
                "outputs": {"categorical_logits": y.name},
                "parameters": {"global_step": global_step.name},
        })
        print(meta)
        tf.add_to_collection("meta", meta)
        filename = "/tmp/lenet_tensorflow.meta"
        tf.train.export_meta_graph(filename, saver_def=saver.as_saver_def())
    return filename

In [8]:
filename = simple_model(28, 28, 3, classes=3)

{"inputs": {"batch_image_input": "Placeholder:0", "categorical_labels": "Placeholder_1:0"}, "outputs": {"categorical_logits": "add:0"}, "parameters": {"global_step": "global_step:0"}}


In [9]:
model = H2ODeepWaterEstimator(epochs=500, 
                              network_definition_file=filename,  ## specify the model
                              image_shape=[28,28],  ## provide expected (or matching) image size
                              channels=3,
                              backend="tensorflow", 
                             ) 
model.train(x=[0], y=1, training_frame=frame)
model.show()

deepwater Model Build progress: |█████████████████████████████████████████| 100%
Model Details
H2ODeepWaterEstimator :  Deep Water
Model Key:  DeepWater_model_python_1490461966315_2


ModelMetricsMultinomial: deepwater
** Reported on train data. **

MSE: 0.0
RMSE: 0.0
LogLoss: 0.0
Mean Per-Class Error: 0.0
Confusion Matrix: vertical: actual; across: predicted



cat,dog,mouse,Error,Rate
90.0,0.0,0.0,0.0,0 / 90
0.0,85.0,0.0,0.0,0 / 85
0.0,0.0,92.0,0.0,0 / 92
90.0,85.0,92.0,0.0,0 / 267


Top-3 Hit Ratios: 


k,hit_ratio
1,1.0
2,1.0
3,1.0


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_classification_error
,2017-03-25 10:13:05,0.000 sec,None,0.0,0,0.0,nan,nan,nan
,2017-03-25 10:13:05,0.118 sec,25600 obs/sec,3.8352060,1,1024.0,0.7520260,19.5331657,0.5655431
,2017-03-25 10:13:08,3.149 sec,43766 obs/sec,502.4119850,131,134144.0,0.0,0.0,0.0


# Custom models with Keras

It is also possible to use libraries/APIs such as Keras to define the network architecture.

In [10]:
import tensorflow as tf
import json
from keras.layers.core import Dense, Flatten, Reshape
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras import backend as K
from keras.objectives import categorical_crossentropy
from tensorflow.python.framework import ops

def keras_model(w, h, channels, classes):
    # always create a new graph inside ipython or
    # the default one will be used and can lead to
    # unexpected behavior
    graph = tf.Graph() 
    with graph.as_default():
        size = w * h * channels
        # Input images fed via H2O
        inp = tf.placeholder(tf.float32, [None, size])
        # Actual labels used for training fed via H2O
        labels = tf.placeholder(tf.float32, [None, classes])

        # Keras network
        x = Reshape((w, h, channels))(inp)
        x = Conv2D(20, (5, 5), padding='same', activation='relu')(x)
        x = MaxPooling2D((2,2))(x)

        x = Conv2D(50, (5, 5), padding='same', activation='relu')(x)
        x = MaxPooling2D((2,2))(x)

        x = Flatten()(x)    

        x = Dense(500, activation='relu')(x)

        out = Dense(classes)(x)

        predictions = tf.nn.softmax(out)

        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels,logits=out))
        train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)

        init_op = tf.global_variables_initializer()

        # Metadata required by H2O
        tf.add_to_collection(ops.GraphKeys.INIT_OP, init_op.name)
        tf.add_to_collection(ops.GraphKeys.TRAIN_OP, train_step)
        tf.add_to_collection("logits", out)
        tf.add_to_collection("predictions", predictions)

        meta = json.dumps({
                "inputs": {"batch_image_input": inp.name,
                           "categorical_labels": labels.name},
                "outputs": {"categorical_logits": out.name,
                            "layers": ','.join([m.name for m in tf.get_default_graph().get_operations()])},
                "parameters": {}
            })
        tf.add_to_collection("meta", meta)

        # Save the meta file with the graph
        saver = tf.train.Saver()
        filename = "/tmp/keras_tensorflow.meta"
        tf.train.export_meta_graph(filename, saver_def=saver.as_saver_def())

        return filename

Using TensorFlow backend.


In [11]:
filename = keras_model(28, 28, 3, classes=3)

In [12]:
model = H2ODeepWaterEstimator(epochs=50, 
                              network_definition_file=filename,  ## specify the model
                              image_shape=[28,28],  ## provide expected (or matching) image size
                              channels=3,
                              backend="tensorflow", 
                             ) 
model.train(x=[0], y=1, training_frame=frame)
model.show()

deepwater Model Build progress: |█████████████████████████████████████████| 100%
Model Details
H2ODeepWaterEstimator :  Deep Water
Model Key:  DeepWater_model_python_1490461966315_3


ModelMetricsMultinomial: deepwater
** Reported on train data. **

MSE: 1.1573397277734483e-07
RMSE: 0.0003401969617403201
LogLoss: 0.00015253441347043935
Mean Per-Class Error: 0.0
Confusion Matrix: vertical: actual; across: predicted



cat,dog,mouse,Error,Rate
90.0,0.0,0.0,0.0,0 / 90
0.0,85.0,0.0,0.0,0 / 85
0.0,0.0,92.0,0.0,0 / 92
90.0,85.0,92.0,0.0,0 / 267


Top-3 Hit Ratios: 


k,hit_ratio
1,1.0
2,1.0
3,1.0


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_classification_error
,2017-03-25 10:13:09,0.000 sec,None,0.0,0,0.0,nan,nan,nan
,2017-03-25 10:13:10,1.070 sec,1091 obs/sec,3.8352060,1,1024.0,0.6121063,0.9657902,0.4419476
,2017-03-25 10:13:11,2.038 sec,7561 obs/sec,53.6928839,14,14336.0,0.0003402,0.0001525,0.0
